<a href="https://colab.research.google.com/github/abvijaykumar/python-lora-finetuning/blob/main/python_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Depedencies

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops accelerate
!pip install -q wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10

In [ ]:
from datasets import load_dataset
from random import randrange

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

from trl import SFTTrainer

from huggingface_hub import login

import wandb

#Initialize the variables

In [ ]:
#model_name = "meta-llama/Llama-2-7b-hf"
#model_name = "TinyPixel/Llama-2-7B-bf16-sharded"
model_name = "Salesforce/codegen-350M-mono"
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
split = "train[:10%]"
finetunes_model_name = "llama-2-7b-finetuned-int4-python-18k-alpaca"
device_map = {"": 0}

In [ ]:
# BitsAndBytesConfig config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="CAUSAL_LM",
)

In [ ]:
def prompt_instruction_format(sample):
  return f"""### Instruction:
    Use the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:

    ### Task:
    {sample['instruction']}

    ### Input:
    {sample['input']}

    ### Response:
    {sample['output']}
    """

#Load the Model & DataSet


In [ ]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

In [ ]:
wandb.login()
%env WANDB_PROJECT=python-fine-tuning

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=python-fine-tuning


In [ ]:
dataset = load_dataset(dataset_name, split=split)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, use_cache = False, device_map=device_map)
base_model.config.pretraining_tp = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#Train the model

In [ ]:
# Define the training arguments
trainingArgs = TrainingArguments(
    output_dir=finetunes_model_name, # Output directory where the model predictions and checkpoints will be stored
    num_train_epochs=3, # Number of training epochs
    per_device_train_batch_size=4, # Batch size per GPU for training
    gradient_accumulation_steps=2,  # Number of update steps to accumulate the gradients for
    gradient_checkpointing=True,# Enable gradient checkpointing
    optim="paged_adamw_32bit", # Optimizer to use
    #save_steps=save_steps,
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    #tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    #max_steps=max_steps,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    report_to="wandb",
    seed=42
)
# Create the trainer
trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:

print("Start the supervised Fine tuning")
trainer.train()
print("Done Training")

#stop reporting to wandb
wandb.finish()

# save model
trainer.save_model()
print("Model saved")

Start the supervised Fine tuning


wandb: Currently logged in as: vijjuk (abvijay). Use `wandb login --relogin` to force relogin


Token indices sequence length is longer than the specified maximum sequence length for this model (2905 > 2048). Running this sequence through the model will result in indexing errors
You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.1543, 'learning_rate': 4.761904761904762e-05, 'epoch': 0.02}
{'loss': 1.0955, 'learning_rate': 9.523809523809524e-05, 'epoch': 0.04}
{'loss': 1.0682, 'learning_rate': 0.00014285714285714287, 'epoch': 0.06}
{'loss': 1.0194, 'learning_rate': 0.00019047619047619048, 'epoch': 0.09}
{'loss': 0.9191, 'learning_rate': 0.00019998282416292055, 'epoch': 0.11}
{'loss': 0.8437, 'learning_rate': 0.00019991305743680013, 'epoch': 1.01}
{'loss': 0.8771, 'learning_rate': 0.00019978966374934254, 'epoch': 1.03}
{'loss': 0.853, 'learning_rate': 0.00019961270933041477, 'epoch': 1.06}
{'loss': 0.7985, 'learning_rate': 0.0001993822891578708, 'epoch': 1.08}
{'loss': 0.7794, 'learning_rate': 0.00019909852690657359, 'epoch': 1.1}
{'loss': 0.7659, 'learning_rate': 0.00019876157488201424, 'epoch': 2.0}
{'loss': 0.7925, 'learning_rate': 0.0001983716139385641, 'epoch': 2.03}
{'loss': 0.7209, 'learning_rate': 0.00019792885338240374, 'epoch': 2.05}
{'loss': 0.7321, 'learning_rate': 0.0001974335308591806, '

train/epoch,▁▁▁▁▁▄▄▄▅▅███████
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇▇██
train/learning_rate,▁▃▅█████████████
train/loss,█▇▇▆▅▄▄▄▃▃▂▃▂▂▁▂
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,2.12
train/global_step,81


Model saved


#Merge the LORA to the main model

In [ ]:
# load the trained model from the output directory
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    trainingArgs.output_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

In [ ]:
# Merge LoRA with the base model and save the merged model
lora_merged_model = trained_model.merge_and_unload()
lora_merged_model.save_pretrained("merged",safe_serialization=True)
tokenizer.save_pretrained("merged")

#push merged model to the hub
lora_merged_model.push_to_hub("codegen-350M-mono-python-18k-alpaca")
tokenizer.push_to_hub("codegen-350M-mono-python-18k-alpaca")

pytorch_model.bin:   0%|          | 0.00/713M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vijjuk/codegen-350M-mono-python-18k-alpaca/commit/af500ed43a1c36f3a2344c9e27f371798e81b8c8', commit_message='Upload tokenizer', commit_description='', oid='af500ed43a1c36f3a2344c9e27f371798e81b8c8', pr_url=None, pr_revision=None, pr_num=None)

#Test the model

In [ ]:
instruction="Write a Python program to generate a Markov chain given a text input."
input="Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'"

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.
### Task:
{instruction}
### Input:
{input}
### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():

print(f"-------------------------\n")
print(f"Prompt:\n{prompt}\n")
print(f"-------------------------\n")

print(f"Base Model Response :\n")
output_base = base_model.generate(input_ids=input_ids, max_new_tokens=500, do_sample=True, top_p=0.9,temperature=0.5)
print(f"{tokenizer.batch_decode(output_base.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n")

print(f"Trained Model Response :\n")
trained_model = lora_merged_model.generate(input_ids=input_ids, max_new_tokens=500, do_sample=True, top_p=0.9,temperature=0.5)
print(f"{tokenizer.batch_decode(trained_model.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n")

print(f"LORA Model Response :\n")
output_trained_lora = lora_merged_model.generate(input_ids=input_ids, max_new_tokens=500, do_sample=True, top_p=0.9,temperature=0.5)
print(f"{tokenizer.batch_decode(output_trained_lora.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------------------------

Prompt:
### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.
### Task:
Write a Python program to generate a Markov chain given a text input.
### Input:
Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'
### Response:


-------------------------

Base Model Response :



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


from random import choice

# Generate a Markov chain
def generate_markov_chain(text):
    words = text.split()
    words_length = len(words)
    words_count = {}
    for word in words:
        if word not in words_count:
            words_count[word] = 1
        else:
            words_count[word] += 1
    words_count = sorted(words_count.items(), key=lambda x: x[1], reverse=True)
    words_count = words_count[:words_length]
    words_list = []
    for word in words_count:
        words_list.append(word[0])
    return words_list

# Generate a Markov chain given a text input
def generate_markov_chain_text(text):
    words = text.split()
    words_length = len(words)
    words_count = {}
    for word in words:
        if word not in words_count:
            words_count[word] = 1
        else:
            words_count[word] += 1
    words_count = sorted(words_count.items(), key=lambda x: x[1], reverse=True)
    words_count = words_count[:words_length]
    words_list = []
    for word in wo

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


import random

def markov_chain(text):
    """
    Generates a Markov chain given a text input
    """
    # Generate Markov chain
    chain = ['']
    for i in range(len(text)):
        # Markov chain
        word = chain[i]
        for j in range(len(text)):
            # Markov chain
            next_word = word + text[j]
            if j == 0:
                next_word = word + '.'
            chain.append(next_word)
    return chain

def generate_text(chain):
    """
    Generates a text given a Markov chain
    """
    text = ''
    for i in range(len(chain)):
        text += chain[i]
    return text

# Generate Markov chain
text = markov_chain('Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?')

# Generate text
text = generate_text

#Cleanup the VRAM

In [ ]:
import gc

# clear the VRAM
del base_model
del trained_model
del lora_merged_model
del trainer
torch.cuda.empty_cache()
gc.collect()

8659